In [ ]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras

In [ ]:
training_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range = 0.12,
    height_shift_range = 0.12,
    width_shift_range = 0.12,
    horizontal_flip = True
)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
import os
import random
all_classes = os.listdir('./data/images/Images')
selected_classes = all_classes[:100]
train_generator = training_data_gen.flow_from_directory('./data/images/Images',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')
test_generator = test_data_gen.flow_from_directory('./data/test',
                                                        target_size=(300, 300),
                                                        batch_size=32,
                                                        class_mode='categorical')

In [ ]:
base_model = keras.applications.MobileNet(
    weights='imagenet',
    input_shape=(300,300, 3),
    include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
inputs = keras.Input(shape=(300, 300, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(100, activation='softmax')(x)

model = Model(base_model.input, predictions)
print(model.summary())

In [ ]:
model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
import scipy.ndimage
early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=1,
    restore_best_weights=True
)
model.fit(train_generator,
                        epochs=10,
                        validation_data=test_generator, callbacks=[early_stopping])

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')

In [ ]:
model.save('NN/sanford_dog_mobilenet.h5')

In [ ]:
from keras.saving.save import load_model

model = load_model('NN/sanford_dog_mobilenet.h5')
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')